# Notify on new imagery in an AOI

Automate the process of checking for newly added catalog imagery over a specified AOI by setting up notifications to your email address.

## Set up the notebook

### 1. Install dependencies

In [ ]:
!pip install up42-py --upgrade -q

import up42, pathlib, json, time, smtplib
import geopandas as gpd
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from shapely.geometry import mapping
from collections import defaultdict

### 2. Configure credentials

Run the cell below to create a `credentials.json` file in a directory named `.up42` in your home folder.

In [ ]:
# Define the credentials file path if it doesn't exist
credentials_file_path = pathlib.Path.home().joinpath(".up42/credentials.json")
credentials_file_path.parent.mkdir(parents=True, exist_ok=True)
credentials_file_path.touch(exist_ok=True)

# Print the path to the file
print(f"Credentials file is located at: {credentials_file_path}")

### 3. Authenticate

In [ ]:
up42.authenticate(cfg_file=credentials_file_path)

### 4. Set up sender credentials

The script is configured to use Gmail's SMTP server, so the sending account must be a Gmail account.

Run the cell below to create a `google_credentials.json` file in a directory named `.up42` in your home folder. The username in this file will be used to log in to the email server to send the notification. It will act as the "FROM" address.

In [ ]:
# Define the credentials file path if it doesn't exist
google_credentials_file_path = pathlib.Path.home().joinpath(".up42/google_credentials.json")
google_credentials_file_path.parent.mkdir(parents=True, exist_ok=True)
google_credentials_file_path.touch(exist_ok=True)

# Print the path to the file
print(f"Credentials file is located at: {google_credentials_file_path}")

GOOGLE_CREDENTIALS_PATH = google_credentials_file_path

1. Click the link above to the created file and paste the following code:
    ```json
    {
        "username": "<your-gmail-address>",
        "password": "<your-password>"
    }
    ```
2. Save the `google_credentials.json` file.

### 5. Specify the receiving email address

Specify the email address where notifications will be sent.

In [ ]:
RECEIVER_EMAIL = "your-email-address@example.com"

## Step 1. Define search parameters

In [ ]:
# List UP42 catalog collections to search from
COLLECTIONS_TO_SEARCH = [
    "pneo-hd15",
    "worldview-legion-hd",
    "beijing-3a",
]

# Specify the maximum cloud cover percentage from 0 to 100
MAX_CLOUD_COVER_PERCENTAGE = 30

# Specify the start date for the search in YYYY-MM-DD format
SEARCH_START_DATE = "2025-09-18"

# Load the AOI
CUSTOMER_AOI_PATH = "/Users/your-path/Desktop/Example.geojson"
customer_aoi = gpd.read_file(CUSTOMER_AOI_PATH)

## Step 2. Run the initial search

The first search establishes a baseline of existing imagery. The monitoring loop will compare future search results against this baseline to find new scenes.

### 1. Identify the host

In [ ]:
# Get all available archive collections
archive_collections = up42.CollectionType.ARCHIVE
sort_by_name = up42.CollectionSorting.name
data_collections = up42.ProductGlossary.get_collections(
    collection_type=archive_collections,
    sort_by=sort_by_name
)

# Create a quick-access map of collection names to their host objects
collection_to_host_map = {}
for c in data_collections:
    try:
        host = next(p for p in c.providers if p.is_host)
        collection_to_host_map[c.name] = host
    except StopIteration:
        pass

# Store the host object and its collections
host_to_collections_map = defaultdict(lambda: {"host_obj": None, "collections": []})

for collection_name in COLLECTIONS_TO_SEARCH:
    if collection_name in collection_to_host_map:
        host = collection_to_host_map[collection_name]
        host_title = host.title # Use the string title as the key

        host_to_collections_map[host_title]["host_obj"] = host
        host_to_collections_map[host_title]["collections"].append(collection_name)
    else:
        print(f"Warning: Collection '{collection_name}' not found or has no host.")

print("Collections grouped by host for searching:")
for host_title, data in host_to_collections_map.items():
    print(f"- Host '{host_title}': {data['collections']}")

### 2. Perform the initial search

In [ ]:
# Get current date
today_date = datetime.today().strftime('%Y-%m-%d')
aoi_geometry_dict = mapping(customer_aoi.geometry[0])

# Perform search
all_initial_scenes = []
for host_title, data in host_to_collections_map.items():
    host = data["host_obj"]
    collections_for_host = data["collections"]

    print(f"Searching host '{host.title}' for collections: {collections_for_host}")
    try:
        scenes = list(host.search(
            collections=collections_for_host,
            intersects=aoi_geometry_dict,
            start_date=SEARCH_START_DATE,
            end_date=today_date,
            query={
                "cloudCoverage": {"lte": MAX_CLOUD_COVER_PERCENTAGE}
            }
        ))
        all_initial_scenes.extend(scenes)
        print(f"Found {len(scenes)} scenes.\n")
    except Exception as e:
        print(f"An error occurred while searching host '{host_title}': {e}")

# Convert combined results to a DataFrame for easier handling
previous_search_results_df = gpd.GeoDataFrame(all_initial_scenes)
print(f"Total of {len(previous_search_results_df)} images found in the initial search across all hosts.")

## Step 3. Set up the notification system

### 1. Define the email function

This function handles the connection to the SMTP server and sends the formatted email. It takes the email credentials, subject, and body as input.

In [ ]:
def send_email_notification(google_credentials, subject, body, receiver_email):
    # Email configuration
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    sender_email = google_credentials['username']
    sender_password = google_credentials['password']

    # Create the MIMEText object
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Connect to the SMTP server and send the email
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(msg)
    except Exception as e:
        print(f"Error sending email: {e}")
    finally:
        server.quit()

### 2. Define the checking function

The cell below performs a new search with the same parameters. It then compares the scene IDs from the new search with the IDs from the previous search. If new IDs are found, it calls the `send_email_notification` function for each new image.

In [ ]:
def check_for_new_imagery(previous_search_results_df, search_params, google_credentials, receiver_email):
    host_map, aoi_dict, start_date, query = search_params
    today_date = datetime.today().strftime('%Y-%m-%d')

    all_recent_scenes = []
    # Loop through each host and search for its collections
    for host_title, data in host_map.items():
        host = data["host_obj"]
        collections = data["collections"]

        scenes = list(host.search(
            collections=collections,
            intersects=aoi_dict,
            start_date=start_date,
            end_date=today_date,
            query=query,
        ))
        all_recent_scenes.extend(scenes)

    most_recent_search_results_df = gpd.GeoDataFrame(all_recent_scenes)

    # Compare results and send notifications
    if len(most_recent_search_results_df) > len(previous_search_results_df):
        initial_search_unique_ids = list(previous_search_results_df['id'])
        most_recent_search_unique_ids = list(most_recent_search_results_df['id'])

        new_ids = [x for x in most_recent_search_unique_ids if x not in initial_search_unique_ids]

        for new_id in new_ids:
            now_time = str(datetime.now())
            subject = "New UP42 imagery alert"
            body = f'New catalog image available over your AOI.\n\nScene ID: {new_id}\nTime found: {now_time}'
            print(f"New imagery found: {new_id}. Sending a notification.")
            send_email_notification(google_credentials, subject, body, receiver_email)
    else:
        now_time = str(datetime.now())
        print(f'No new imagery found over AOI at {now_time}')

    return most_recent_search_results_df

## Step 4. Run the monitoring loop

This final cell starts an infinite loop to monitor for new imagery. It will run the `check_for_new_imagery` function, wait for a specified interval, and repeat.

To stop the script, interrupt the kernel.

In [ ]:
# Load credentials
with open(GOOGLE_CREDENTIALS_PATH) as f:
    google_credentials = json.load(f)

# Convert the AOI geometry to a JSON-serializable dictionary
aoi_geometry_dict = mapping(customer_aoi.geometry[0])

# Define search parameters tuple for the checking function
search_params = (
    host_to_collections_map, # Pass the grouped hosts and collections
    aoi_geometry_dict,
    SEARCH_START_DATE,
    {"cloudCoverage": {"lte": MAX_CLOUD_COVER_PERCENTAGE}}
)

# Check for new imagery periodically
while True:
    try:
        most_recent_search_results_df = check_for_new_imagery(
            previous_search_results_df,
            search_params,
            google_credentials,
            RECEIVER_EMAIL
        )
        previous_search_results_df = most_recent_search_results_df

        # Sleep for 3600 seconds (1 hour)
        print("Check complete. Waiting for 1 hour before the next check.")
        time.sleep(3600)

    except KeyboardInterrupt:
        print("\nMonitoring stopped by user.")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        # Add a shorter sleep time on error before retrying
        time.sleep(300)